In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

DIAGS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I350",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J398",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J90X",
    "J91X",
    "J931",
    "J955",
    "J980",
    "M348",
    "Q211",
    "Q231",
    "Q676",
    "T820",
    "T821",
    "Z450",
]

DIAGS_FALTANTES_PULMON = ["C349", "C348"]

DIAGS_RELEVANTES += DIAGS_FALTANTES_PULMON

In [3]:
# Importa los casos teoricos por cada problema de salud
casos_teoricos = pd.read_excel("../data/interim/casos_teoricos_diagnosticos.xlsx")
casos_nacionales = casos_teoricos.query("Estrato == 'Pais'").copy()

In [4]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv", sep=";", encoding="latin-1"
)

In [12]:
egresos_mas_relevantes = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017")
    .copy()
)

In [15]:
resumen_egresos_mas_relevantes = pd.pivot_table(
    egresos_mas_relevantes,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_pacientes_distintos", "n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [16]:
resumen_egresos_mas_relevantes

dias_estada_totales                      n_egresos              \
ANO_EGRESO                2017   2018   2019   2020      2017  2018  2019   
DIAG1                                                                       
C33X                       426    193    155    136        25    18    17   
C340                      1217   1141   1163    661       163   143   148   
C341                      3682   4267   4425   2514       372   478   493   
C342                       494    734    394    478        50    68    52   
C343                      2376   2522   2446   1798       258   293   279   
C348                       367    256    358    350        35    47    44   
C349                     26058  25323  21719  18126      2492  2510  2475   
C381                       474    403    575    319        41    41    52   
C384                       612    449    508    313        39    44    52   
C450                       661    566    560    396        81    76    75   
C780                      3973   3348   3532   2886       373   379   430   
C782                      1691   1839   2130   1835       147   188   211   
D143                      1286   1222    762    537       178   207   160   
D381                      4665   5365   5354   3964       553   641   654   
E848                       891   1188    725    606        58    78    56   
I080                      1552   1907   1578    865        96   120   110   
I081                       843   1461   1063    457        51    67    60   
I340                      6719   6304   4366   3614       436   413   341   
I350                     16427  18300  17666  12946      1359  1481  1567   
I351                      4155   4062   3698   2788       318   327   313   
I352                      1727   1787   1733   1326        96   126   139   
I420                      5524   5953   6417   3839       489   497   561   
I456                       513    551    504    371       202   190   183   
I472                      3474   3521   3925   2725       429   478   442   
I495                      5499   6143   6091   4860       892  1077  1147   
I710                      5076   6138   6098   5509       429   474   467   
I712                      1371   1519   1305   1582       109   130   122   
J398                      2352    522    829   1721       124    92   106   
J47X                      3941   3747   4168   1938       377   373   395   
J679                      1171   1080   1235    354       106   101   109   
J841                     16577  15087  15943  10135      1666  1633  1697   
J848                      1098   1433   1596   1158       113   172   158   
J849                      4126   5113   5342   2989       502   564   600   
J860                      1535   1436   1041    895        66    95    49   
J869                      9494   7948   8036   5712       417   372   408   
J90X                     24326  25800  27836  20457      2163  2252  2424   
J91X                         0     18      0      0         0     3     0   
J931                      4007   3946   3855   2788       475   481   443   
J955                      1628   2770   1372   1525       167   162   167   
J980                       565    671    625    352       112   105    88   
M348                       432    435    596     93        44    55    45   
Q211                      2208   1918   1565   1374       396   366   409   
Q231                        91    243    217    622        19    29    27   
Q676                       650    745    845    419       149   154   131   
T820                       515    341    700    912        35    41    50   
T821                      1193    881   1134    810       180   167   222   
Z450                      1530   1436   1470    930       718   649   685   

                 n_pacientes_distintos                    
ANO_EGRESO  2020                  2017  2018  2019  2020  
DIAG1                                     